<a href="https://colab.research.google.com/github/GowshikPrabhu/flutter-fandvrecognizer/blob/master/FruitsClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wget

In [2]:
import tensorflow as tf
import os
import wget

In [3]:
URL = 'https://github.com/Horea94/Fruit-Images-Dataset/archive/master.zip'

In [4]:
wget.download(URL)

'Fruit-Images-Dataset-master.zip'

In [ ]:
!unzip Fruit-Images-Dataset-master.zip;

In [6]:
!mv Fruit-Images-Dataset-master fruits-360

In [7]:
base_dir = '/content/fruits-360/Training'

In [20]:
tf.debugging.set_log_device_placement(True)

In [9]:
IMAGE_SIZE = 244
BATCH_SIZE = 64
with tf.device('/GPU:0'):
    datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255, 
        rotation_range=30,
        width_shift_range=0.3,
        height_shift_range=0.3,
        horizontal_flip=True,
        validation_split=0.1,
        fill_mode='nearest'
    )

    training_generator = datagen.flow_from_directory(
        base_dir,
        target_size=(IMAGE_SIZE, IMAGE_SIZE),
        batch_size=BATCH_SIZE,
        subset='training'
    )

Found 60955 images belonging to 131 classes.


In [10]:
labels = '\n'.join(sorted(training_generator.class_indices.keys()))

with open('labels.txt', 'w') as f:
    f.write(labels)

In [ ]:
IMAGE_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMAGE_SHAPE,
    alpha=1.0,
    include_top=False,
    weights='imagenet'
)

In [13]:
base_model.trainable = False

In [ ]:
model = tf.keras.Sequential([
                             base_model,
                             tf.keras.layers.Conv2D(32, 3, activation='relu'),
                             tf.keras.layers.Dropout(0.2),
                             tf.keras.layers.GlobalAveragePooling2D(),
                             tf.keras.layers.Dense(131, activation='softmax')
])

In [ ]:
with tf.device('/GPU:0'):
    model.compile(
        tf.keras.optimizers.Adam(),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

In [ ]:
epochs = 10
with tf.device('/GPU:0'):
    history = model.fit(training_generator, epochs=epochs, steps_per_epoch=200)

In [ ]:
saved_model_dir = ''
tf.saved_model.save(model, saved_model_dir)

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

In [23]:
from google.colab import files

files.download('model.tflite')
files.download('labels.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>